In [2]:
# %load eval.py
from argparse import ArgumentParser

from pytorch_lightning import seed_everything, Trainer

from dataset.mpii_face_gaze_dataset import get_dataloaders
from train import Model

if __name__ == '__main__':

    ###非變動區塊
    k = [9, 128]
    adjust_slope = False
    ###對應作者表格grid calibration,如果是random就填true
    grid_calibration_samples=False
    batch_size =32
    path_to_data = './data/mpiifacegaze_preprocessed'
    
    
    ####變動區塊
    path_to_checkpoints = './saved_models/p00/p00_best-v22.ckpt'
    person_idx = 1
    validate_on_person = 0 
    
    seed_everything(42)

    print(f"{path_to_checkpoints}")
    
    model = Model.load_from_checkpoint(path_to_checkpoints, k=[9, 128], adjust_slope=False, grid_calibration_samples=False)

    trainer = Trainer(
        gpus=1,
        benchmark=True,
    )

    _, _, test_dataloader = get_dataloaders(path_to_data, validate_on_person, person_idx, batch_size)
    trainer.test(model, test_dataloader)


Global seed set to 42


./saved_models/p00/p00_best-v22.ckpt


C:\Users\User\anaconda3\envs\gazegpu\lib\site-packages\pytorch_lightning\loops\utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


train on persons [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
valid on person 0
test on person 1
len(dataset_train) 60784
len(dataset_valid) 2927


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


len(dataset_test) 2904
Testing DataLoader 0: 100%|██████████| 181/181 [00:15<00:00, 11.60it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             Test metric                         DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test/offset(k=0)/angular_error             5.6869916915893555
        test/offset(k=0)/loss                0.005451828707009554
test/offset(k=128)/mean_angular_error         2.2585714012145996
test/offset(k=128)/std_angular_error         0.025865651074331485
 test/offset(k=9)/mean_angular_error          2.3921381779432296
 test/offset(k=9)/std_angular_error           0.16531519518077176
  test/offset(k=all)/angular_error             2.247389793395996
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
